In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
import numpy as np

# Step 1: Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        target = self.targets[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, target

seed = 12
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Step 2: Define transformations for your custom dataset
transform = T.Compose([
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip(),
        T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.10),
        T.RandomAffine(degrees=(-30, 30), translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])


# Step 3: Define function to get data loaders for train, validation, and test datasets
def get_data_loaders(data_dir, test_dir, batch_size):
    train_data = datasets.ImageFolder(os.path.join(data_dir), transform=transform)
    test_data = datasets.ImageFolder(os.path.join(test_dir), transform=test_transform)

    train_size = int(0.8 * len(train_data))
    val_size = len(train_data) - train_size
    train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shu kiffle=False, num_workers=4)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

import timm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Step 4: Load pre-trained PyTorch models
model1 = timm.create_model('deit_base_patch16_224.fb_in1k', pretrained=True)
model2 = timm.create_model('densenet169', pretrained=True)



# Modify the models for binary classification
import torch
import torch.nn as nn
import torch.nn.init as init

# Assuming 'model' is your existing model
# Freeze all parameters in the existing model
for param in model1.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model1.head.in_features

# Modify the fc layer and add more layers for reducing val loss
n_inputs = model1.head.in_features
model1.head = nn.Sequential(
    nn.Linear(n_inputs, 2048),
    nn.BatchNorm1d(2048),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.2),  # Dropout for regularization

    nn.Linear(2048, 1024),  # Increase the number of neurons
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(64, 6)
)

model1 = model1.to(device)


import torch
import torch.nn as nn
import torch.nn.init as init

# Assuming 'model' is your existing model
# Freeze all parameters in the existing model
for param in model2.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model2.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model2.classifier = nn.Sequential(
      nn.Linear(n_inputs, 2048),
    nn.BatchNorm1d(2048),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.2),  # Dropout for regularization

    nn.Linear(2048, 1024),  # Increase the number of neurons
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(64, 6)
)

model2 = model2.to(device)




# Step 5: Load pre-trained PyTorch models
model1_path = "/kaggle/input/10x-deigset/models/DeiT/model_weights_epoch_199.pth"
model2_path = "/kaggle/input/10x-deigset/models/DenseNet169/model_weights_epoch_199_DenseNet169.pth"
#model4_path = "/kaggle/input/10x-deigset/models/pvt/model_weights_epoch_199_pvt.pth"



model1.load_state_dict(torch.load(model1_path))
model2.load_state_dict(torch.load(model2_path))

#model4.load_state_dict(torch.load(model4_path))

# Set models to evaluation mode
model1.eval()
model2.eval()

#model4.eval()


# Step 6: Make predictions using the pre-trained models
def get_predictions(model, dataloader):
    all_predictions = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)  # Move inputs to the appropriate device
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())  # Convert to numpy array for stacking
    return all_predictions

# Step 7: Stack the predictions for meta-model training
train_loader, val_loader, test_loader = get_data_loaders("/kaggle/input/10x-diaga-1200/10x/", "/kaggle/input/diag-10x-500/10X/Test/", batch_size=128)

# Get predictions for training and validation sets
predictions_model1_train = get_predictions(model1, train_loader)
predictions_model2_train = get_predictions(model2, train_loader)


stacked_predictions_train = np.column_stack((predictions_model1_train, predictions_model2_train))

# Get predictions for validation set for training the meta-model
predictions_model1_val = get_predictions(model1, val_loader)
predictions_model2_val = get_predictions(model2, val_loader)



stacked_predictions_val = np.column_stack((predictions_model1_val, predictions_model2_val))

# Extract labels for the validation set
val_labels = []
for _, target in val_loader.dataset:
    val_labels.append(target)
val_labels = np.array(val_labels)

# Train the logistic regression meta-model
meta_model = SVC() 
#meta_model = GradientBoostingClassifier() 
#meta_model = DecisionTreeClassifier()
#meta_model = LogisticRegression(), 
#meta_model = KNeighborsClassifier() 
#meta_model = AdaBoostClassifier() 
#meta_model = XGBClassifier()
#meta_model = GaussianNB() 
#meta_model = GradientBoostingClassifier()  
#meta_model = RandomForestClassifier()
meta_model.fit(stacked_predictions_val, val_labels)

# Step 9: Make predictions on the test dataset
test_predictions_model1 = get_predictions(model1, test_loader)
test_predictions_model2 = get_predictions(model2, test_loader)


test_stacked_predictions = np.column_stack((test_predictions_model1, test_predictions_model2))

# Extract labels for the test set
test_labels = []
for _, target in test_loader.dataset:
    test_labels.append(target)
test_labels = np.array(test_labels)

# Step 10: Use the meta-model to make predictions on the stacked test predictions
test_meta_preds = meta_model.predict(test_stacked_predictions)

# Step 11: Evaluate the accuracy of the stacked ensemble model
ensemble_accuracy = accuracy_score(test_labels, test_meta_preds)
print("Stacked Ensemble Accuracy:", ensemble_accuracy)